###  <font color='green' size=5> HW #6: Air Quality - Cassandra  </font> 

**NoSQL** <br>
**ANLT 243** <br>
**September 22, 2018** <br>
<br>

<font color='green' size=4> By: Ali Taheri</font>

At first, we prepare environment for access to database:

In [8]:
from cassandra.cluster import Cluster
import json

host = 'ec2-54-67-15-68.us-west-1.compute.amazonaws.com'
port = 9220

cluster = Cluster([host], port = port)

session = cluster.connect()

session.set_keyspace('nosql')

##  <font color='blue' size=4> 1. How many total records are there in this air_quality table ?  </font>

In [9]:
#counting number of records by aggregate function of count
rows = session.execute('Select count(measureid) from air_quality')

for row in rows:
    print('Number of records in table of air quality is:', row.system_count_measureid)

Number of records in table of air quality is: 218635


##  <font color='blue' size=4> 2. What was the air quality metric value for MeasureId 83 in the Sandoval county in the state of New Mexico in 2013 ?  </font>



In [10]:
county_stmt = session.prepare("Select * from air_quality where measureid=? and countyname=? and statename=? and reportyear=?")
rows = session.execute(county_stmt, [83, 'Sandoval','New Mexico',2013])

for row in rows:
    print(' The air quality metric value for MeasureId 83 in the Sandoval county in the state of New Mexico in 2013 is:', row.value)


 The air quality metric value for MeasureId 83 in the Sandoval county in the state of New Mexico in 2013 is: 0


##  <font color='blue' size=4> 3.Which state and county have the highest recorded air quality value of MeasureId 87 in any year ?  </font>


At first, we find global maximum of this measureid:

In [11]:
rows = session.execute('Select max(value) from air_quality where measureid=87')
for row in rows:
    print('The max of 87 measureid is :', row.system_max_value)

The max of 87 measureid is : 51


At this point, we try to find county and state with this value of 51 for measureid of 87.

In [12]:
rows = session.execute("Select countyname, statename, reportyear, value from air_quality where measureid=87 and value=51")
for row in rows:
     print( row.countyname + " from "+ row.statename + " had the maximum value of "+ \
             str(row.value) + " in " + str(row.reportyear))

Fairbanks North Star from Alaska had the maximum value of 51 in 2004


Now, we want to find maximum for each year. We could see that private key has 4 columns and reportyear is last one,
se we need to create index over it.

In [13]:
#session.execute("CREATE INDEX ON air_quality (reportyear)")

Now, we find starting and ending year of report.

In [14]:
from_to_years = session.execute("Select min(reportyear) , max(reportyear) from air_quality where measureid=87")
for row in from_to_years:
    from_year = row.system_min_reportyear
    to_year = row.system_max_reportyear
    print("Reports of measureid of 87 are from "+ str(from_year) +" to " + str(to_year))

Reports of measureid of 87 are from 1999 to 2013


Finally in a for loop we find maximum value for each year and then locate all records with that value in that year.
Also, we delete duplicated ones to make a better report.

In [15]:
for year in range(from_year,to_year+1):
    rows_max_stmt = session.prepare("Select max(value) from air_quality where measureid= ? and reportyear= ?")
    rows_max = session.execute(rows_max_stmt,[87,year])
    
    for row in rows_max:
         max_value_of_87_this_year = row.system_max_value
        
    county_stmt = session.prepare("Select * from air_quality where measureid=? and value=?")
    rows= session.execute(county_stmt,[87,max_value_of_87_this_year])
    
    old_county = None
    new_state = None
    
    for row in rows:
        new_county , new_state = row.countyname , row.statename
        
        if (new_county != old_county) or (new_state != old_state) :
              print( row.countyname + " from "+ row.statename + " had the maximum value of "+ \
             str(max_value_of_87_this_year) + " in " + str(year))
                
        old_county , old_state = row.countyname , row.statename
            
      
    

Riverside from California had the maximum value of 30 in 1999
Riverside from California had the maximum value of 28 in 2000
Riverside from California had the maximum value of 30 in 2001
Fresno from California had the maximum value of 27 in 2002
Tulare from California had the maximum value of 27 in 2002
Riverside from California had the maximum value of 27 in 2002
Stanislaus from California had the maximum value of 24 in 2003
Los Angeles from California had the maximum value of 24 in 2003
Kern from California had the maximum value of 24 in 2003
Los Angeles from California had the maximum value of 24 in 2003
Riverside from California had the maximum value of 24 in 2003
Scioto from Ohio had the maximum value of 24 in 2003
Fairbanks North Star from Alaska had the maximum value of 51 in 2004
Morgan from Alabama had the maximum value of 21 in 2005
Russell from Alabama had the maximum value of 21 in 2005
Kern from California had the maximum value of 21 in 2005
Fresno from California had the m

Clay from Alabama had the maximum value of 16 in 2011
Escambia from Alabama had the maximum value of 16 in 2011
Madison from Alabama had the maximum value of 16 in 2011
Shelby from Alabama had the maximum value of 16 in 2011
Tuscaloosa from Alabama had the maximum value of 16 in 2011
Russell from Alabama had the maximum value of 16 in 2011
Fairbanks North Star from Alaska had the maximum value of 16 in 2011
Santa Cruz from Arizona had the maximum value of 16 in 2011
Pulaski from Arkansas had the maximum value of 16 in 2011
Santa Clara from California had the maximum value of 16 in 2011
Yolo from California had the maximum value of 16 in 2011
Imperial from California had the maximum value of 16 in 2011
Kings from California had the maximum value of 16 in 2011
Merced from California had the maximum value of 16 in 2011
San Joaquin from California had the maximum value of 16 in 2011
Kings from California had the maximum value of 16 in 2011
Fresno from California had the maximum value of 16

Madison from Alabama had the maximum value of 15 in 2012
Colbert from Alabama had the maximum value of 15 in 2012
Houston from Alabama had the maximum value of 15 in 2012
Mobile from Alabama had the maximum value of 15 in 2012
Sumter from Alabama had the maximum value of 15 in 2012
Russell from Alabama had the maximum value of 15 in 2012
Talladega from Alabama had the maximum value of 15 in 2012
Etowah from Alabama had the maximum value of 15 in 2012
Russell from Alabama had the maximum value of 15 in 2012
Etowah from Alabama had the maximum value of 15 in 2012
Russell from Alabama had the maximum value of 15 in 2012
Jefferson from Alabama had the maximum value of 15 in 2012
Crittenden from Arkansas had the maximum value of 15 in 2012
Pulaski from Arkansas had the maximum value of 15 in 2012
Imperial from California had the maximum value of 15 in 2012
Butte from California had the maximum value of 15 in 2012
San Diego from California had the maximum value of 15 in 2012
San Joaquin from

Jefferson from Alabama had the maximum value of 22 in 2013
Pinal from Arizona had the maximum value of 22 in 2013
Kern from California had the maximum value of 22 in 2013
Tulare from California had the maximum value of 22 in 2013
Los Angeles from California had the maximum value of 22 in 2013
Riverside from California had the maximum value of 22 in 2013
Kern from California had the maximum value of 22 in 2013


##  <font color='blue' size=4> 4-Print the average of air quality value and the unit for MeasureId 84, State - Massachusetts, CountyName - Barnstable, for years between 2002 - 2008 ?  </font>

In [16]:
rows = session.execute("Select avg(value) , unitname from air_quality where measureid = 84 and statename = 'Massachusetts' \
                       and countyname ='Barnstable' and reportyear in (2002,2003,2004,2005,2006,2007,2008) " )                     
for row in rows:
    print('The average value of air quality value and the unit for MeasureId 84,\
    State - Massachusetts, CountyName - Barnstable, for years between 2002 - 2008 is :', \
          row.system_avg_value , row.unitname)

The average value of air quality value and the unit for MeasureId 84,    State - Massachusetts, CountyName - Barnstable, for years between 2002 - 2008 is : 2607774 No Units
